<a href="https://colab.research.google.com/github/maryantonopoulou/teliki_ergasia_EDDE2/blob/main/Athens_voice_articles_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

import time
import random

from tqdm import tqdm

In [ ]:
def get_webpage_soup(url):
  headers = {
        "User-Agent": "Mozilla/5.0"
    }
  response = requests.get(url, headers=headers)
  if response.status_code != 200:
    return None
  else:
    # print(f"{url} was scraped successfully")
    return BeautifulSoup(response.text, 'html.parser')

In [ ]:
def get_article_elements_from_soup(soup, article_url):
  # απομόνωση του block με το περιεχόμενο του άρθρου από όλη τη σούπα της σελίδας
  article_block = soup.find("div", {"class": "articleSingle articleSingle--default"})
  # article url
  article_url = article_url
  # title
  try:
    title = article_block.find("h1", {"class": "articleSingle__title"}).text.strip()
  except:
    # try:
    #   title = soup.find("div", {"class": "container is-fullhd p-3"}).h1.text.strip() # προσθήκη για να πιάνει και τον τίτλο όταν βρίσκεται σε αυτό το div
    # except:
    title=""
  # datetime
  try:
    datetime_str = article_block.find("time")["datetime"]
  except:
    try:
      datetime_str = soup.find("li", {"class": "actionsList__item actionsList__item--date"}).find("time")["datetime"]
    except:
      datetime_str = ""
      date_str = ""
      time_str = ""
  if datetime_str != "":
    date_str = datetime_str.split("T")[0]
    time_str = datetime_str.split("T")[1].split("Z")[0]
  # section
  try:
    section = article_block.find("a", class_=lambda c: c and "articleSingle__category" in c).text.strip()
  except:
    section = ""
  # author
  try:
    author = article_block.find("a", class_=lambda c: c and "actionsList__link" in c).find("span").text.strip()
  except:
    # try:
    #   author = soup.find("div", {"class": "container is-fullhd p-3"}).find("a", {"rel": "author"}).text.strip() # προσθήκη
    # except:
    author = ""
  #subtitle
  try:
    subtitle = article_block.find("div", {"class": "articleSingle__intro"}).text.strip()
  except:
    subtitle = ""
  # excerpt
  try:
    excerpt = article_block.find("div", {"class": "articleSingle__summaryContent"}).find("p").text.strip()
  except:
    # try:
    #   excerpt = soup.find("div", {"class": "container is-fullhd p-3"}).find("div", {"class": "newsletter-description nx-excerpt my-5"}).text.strip() # προσθήκη
    # except:
    excerpt=""
  # text body
  try:
    text_body_p_tagsL = article_block.find("div", class_="articleSingle__description").find_all("p")
    text_body_textsL = []
    for p_tag in text_body_p_tagsL:
      p_tag_text = p_tag.text.strip()
      text_body_textsL.append(p_tag_text)
    text_body = " ".join(text_body_textsL)
  except:
    text_body=""
  # feature image
  try:
    feat_img_src = article_block.find("div", class_="articleSingle__image").find('img')["src"]
    feat_img_link = "https://www.athensvoice.gr"+feat_img_src
  except:
    feat_img_link=""


  # Δημιουργία λεξικού με τα data του άρθρου
  articleD = {"article_url": article_url, "title": title, "date": date_str, "time": time_str, "section": section, "author": author,
              "excerpt": excerpt, "subtitle": subtitle, "text_body": text_body, "feat_img_link": feat_img_link}

  return articleD

In [ ]:
def scrape_athensvoice_articles_pages(teasers_df):
  all_articles_dataL = []
  for idx, row in tqdm(teasers_df.iterrows(), total=teasers_df.shape[0], desc="Processing rows"):

    article_url = row['article_url']
    soup = get_webpage_soup(article_url)
    if soup is not None:
      articleD = get_article_elements_from_soup(soup, article_url)
      all_articles_dataL.append(articleD)
    else:
      # Optional: Print or log the URL that failed to scrape
      print(f"Warning: Could not scrape {article_url}")
    # articleD = get_article_elements_from_soup(soup, article_url)
    # all_articles_dataL.append(articleD)
    delay = random.uniform(1, 3)
    time.sleep(delay)
  df = pd.DataFrame(all_articles_dataL)
  print(f"\nScraping completed. Fetched {len(df)} articles.")
  return df

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
teasers_df = pd.read_csv('/content/drive/MyDrive/CSVFILES/ergatika_atiximataavdf.csv')
teasers_df.head()

,title,article_url,date,time,author,excerpt,section
0,Τροχαίο ατύχημα με σοβαρά τραυματισμένη μια 17...,https://www.athensvoice.gr/epikairotita/ellada...,2025-06-03,07:44:02,Newsroom,Τροχαίο ατύχημα με σοβαρά τραυματισμένη μια 17...,Ελλαδα
1,Τροχαίο ατύχημα στην Πανεπιστημίου - Σοβαρά τρ...,https://www.athensvoice.gr/epikairotita/ellada...,2025-06-03,07:39:27,Newsroom,Τροχαίο στην Πανεπιστημίου με σοβαρά τραυματισ...,Ελλαδα
2,Πάτρα: Προαστιακός πέρασε με τις μπάρες ανοιχτ...,https://www.athensvoice.gr/epikairotita/ellada...,2025-06-03,05:16:55,Newsroom,Πάτρα: Προαστιακός πέρασε με τις μπάρες ανοιχτ...,Ελλαδα
3,Νάντια Κοντογεώργη για το ατύχημα της: Έπρεπε ...,https://www.athensvoice.gr/epikairotita/showbi...,2025-06-02,06:38:17,Newsroom,Τι είπε η Νάντια Κοντογεώργη για την περιπέτει...,Showbiz
4,Τροχαίο ατύχημα στην άνοδο της Συγγρού το πρωί...,https://www.athensvoice.gr/epikairotita/ellada...,2025-06-02,04:35:26,Newsroom,Τροχαίο ατύχημα στην άνοδο της Συγγρού το πρωί...,Ελλαδα


In [ ]:
articles_df = scrape_athensvoice_articles_pages(teasers_df)
articles_df

Processing rows: 100%|██████████| 1416/1416 [1:21:10<00:00,  3.44s/it]



Scraping completed. Fetched 1416 articles.


,article_url,title,date,time,section,author,excerpt,subtitle,text_body,feat_img_link
0,https://www.athensvoice.gr/epikairotita/ellada...,Τροχαίο ατύχημα με σοβαρά τραυματισμένη μια 17...,2025-06-03,07:44:02,Ελλαδα,Newsroom,Τροχαίο ατύχημα με σοβαρά τραυματισμένη μια 17...,Εκτοξεύτηκε από τη μηχανή της και κατέληξε κάτ...,Σοβαρό τροχαίο ατύχημα σημειώθηκε λίγο μετά τα...,https://www.athensvoice.gr/images/1074x600/jpg...
1,https://www.athensvoice.gr/epikairotita/ellada...,Τροχαίο ατύχημα στην Πανεπιστημίου - Σοβαρά τρ...,2025-06-03,07:39:27,Ελλαδα,Newsroom,Τροχαίο στην Πανεπιστημίου με σοβαρά τραυματισ...,Τι αναφέρουν οι πρώτες πληροφορίες,Σοβαρό τροχαίο ατύχημα σημειώθηκε τα ξημερώματ...,https://www.athensvoice.gr/images/1074x600/png...
2,https://www.athensvoice.gr/epikairotita/ellada...,Πάτρα: Προαστιακός πέρασε με τις μπάρες ανοιχτ...,2025-06-03,05:16:55,Ελλαδα,Newsroom,Πάτρα: Προαστιακός πέρασε με τις μπάρες ανοιχτ...,Τι ισχυρίζεται ο ΟΣΕ,Ένα σοβαρό περιστατικό σημειώθηκε το μεσημέρι ...,https://www.athensvoice.gr/images/1074x600/jpg...
3,https://www.athensvoice.gr/epikairotita/showbi...,Νάντια Κοντογεώργη για το ατύχημα της: Έπρεπε ...,2025-06-02,06:38:17,Showbiz,Newsroom,Τι είπε η Νάντια Κοντογεώργη για την περιπέτει...,Τι είπε η ηθοποιός για την περιπέτεια υγείας π...,Η Νάντια Κοντογεώργη βρέθηκε καλεσμένη σε τηλε...,https://www.athensvoice.gr/images/1074x600/jpg...
4,https://www.athensvoice.gr/epikairotita/ellada...,Τροχαίο ατύχημα στην άνοδο της Συγγρού το πρωί...,2025-06-02,04:35:26,Ελλαδα,Newsroom,Τροχαίο ατύχημα στην άνοδο της Συγγρού το πρωί...,Δυσκολίες στην κίνηση,Σοβαρό τροχαίο ατύχημα συνέβη νωρίς το πρωί στ...,https://www.athensvoice.gr/images/1074x600/jpg...
...,...,...,...,...,...,...,...,...,...,...
1411,https://www.athensvoice.gr/life/8508/ekpaideys...,Διάσημα πτυχία,2010-09-15,14:12:00,Life,Κατερίνα Παναγοπούλου,,Ποιο μέλος των Rolling Stones συνέχισε και μετ...,Ποιο μέλος των Rolling Stones συνέχισε και μετ...,https://www.athensvoice.gr/images/1074x600/3/j...
1412,https://www.athensvoice.gr/politismos/eikastik...,Παράλληλο ερωτικό σύμπαν,2010-01-13,15:00:34,Εικαστικα,A.V. Team,,,Με κύριο θέμα το απίθανο της ερωτικής συνάντησ...,https://www.athensvoice.gr/images/1074x600/3/J...
1413,https://www.athensvoice.gr/epikairotita/politi...,O ατυχής ελληνοτουρκικός πόλεμος του 2007,2007-03-29,13:45:00,Πολιτικη & Οικονομια,Άγγελος Τσέκερης - Γιώργος Κυρίτσης,,"Mε ένα μετρημένο και σοβαρό παιχνίδι, η εθνική...","Mε ένα μετρημένο και σοβαρό παιχνίδι, η εθνική...",https://www.athensvoice.gr/images/1074x600/3/j...
1414,https://www.athensvoice.gr/epikairotita/athlit...,Kωπηλασία: το άθλημα που έμαθα,2004-09-01,15:00:00,Αθλητισμος,A.V. Guest,,Σκιφ Σικ,Όλο και κάτι μαθαίνεις σε αυτή τη ζωή. Ξεκινάς...,https://www.athensvoice.gr/images/1074x600/3/j...


In [ ]:
articles_df.to_csv('/content/drive/MyDrive/CSVFILES/ergatika_atiximatasthensvoicedf.csv', index=False)